In [1]:
import os,shutil,random

In [2]:
def make_data(filename):
    data = os.listdir(filename)
    x = os.listdir(filename+'/'+data[0])
    y = os.listdir(filename+'/'+data[1])
    random.shuffle(x)
    random.shuffle(y)
    x1 = x[:int(len(x)*0.8)]
    x_ = x[int(len(x)*0.8):]
    y1 = y[:int(len(y)*0.8)]
    y_ = y[int(len(y)*0.8):]
    if os.path.exists('train') is False:
        os.makedirs('train/'+data[0])
        os.makedirs('train/'+data[1])
        [shutil.copyfile(filename+'/'+data[0]+'/'+i,'train'+'/'+data[0]+'/'+i) for i in x1]
        [shutil.copyfile(filename+'/'+data[1]+'/'+i,'train'+'/'+data[1]+'/'+i) for i in y1]
    if os.path.exists('test') is False:
        os.makedirs('test/'+data[0])
        os.makedirs('test/'+data[1])
        [shutil.copyfile(filename+'/'+data[0]+'/'+i,'test'+'/'+data[0]+'/'+i) for i in x_]
        [shutil.copyfile(filename+'/'+data[1]+'/'+i,'test'+'/'+data[1]+'/'+i) for i in y_]

In [30]:
# make_data('picture')

In [3]:
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import MaxPooling2D,Dropout,Flatten,Dense,Activation,Convolution2D
from keras.models import Sequential

Using TensorFlow backend.


In [3]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(150, 150,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(150, 150,...)`
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`
  # Remove the CWD from sys.path while we load stuff.


In [4]:
model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 64)        18496     
__________

In [4]:
train_generator = ImageDataGenerator(rotation_range=5, 
                                     horizontal_flip=True, 
                                     rescale=1./255,
                                     vertical_flip=True)
test_generator = ImageDataGenerator(rescale=1./255)

In [5]:
train = train_generator.flow_from_directory('train',
                                            target_size=(150,150),
                                            batch_size=32,
                                            classes =['Parasitized','Uninfected'],
                                            class_mode='binary')
test = test_generator.flow_from_directory('test',
                                         target_size=(150,150),
                                         batch_size=32,
                                         classes=['Parasitized','Uninfected'],
                                         class_mode='binary')

Found 22047 images belonging to 2 classes.
Found 5511 images belonging to 2 classes.


In [16]:
#训练网络，现在网络已经训练好了
# model.fit_generator(train,
#                    steps_per_epoch=22047/32,
#                    epochs=5,
#                    validation_data=test,
#                    nb_val_samples=800)
# model.save('model.h5')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  """
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., epochs=5, validation_data=<keras_pre..., steps_per_epoch=688.96875, validation_steps=800)`
  """


Epoch 1/5
  3/688 [..............................] - ETA: 15:04 - loss: 0.0837 - acc: 0.9792

KeyboardInterrupt: 

In [6]:
from keras.preprocessing import image
import numpy as np
def test_pic():
    base_dir= 'picture'
    next_dir = os.listdir(base_dir)
    test = []
    for i in range(5):
        for i in next_dir:
            img = image.load_img(base_dir+'/'+i+'/'+random.choice(os.listdir(base_dir+'/'+i)),target_size=(150,150))
            test.append(img)
    return [np.expand_dims(image.img_to_array(i),axis=0) for i in test]
test_ = test_pic()    
random.shuffle(test_)

In [7]:
from keras.models import load_model
model = load_model('model.h5')
labels = (train.class_indices)
labels = dict((v,k) for k,v in labels.items())
for i in test_:
    pred1 = model.predict_classes(i)
# pred = model.predict_generator(test,verbose=1,steps=3)
# print(pred)
# predicted_class_indices=np.argmax(pred,axis=1)
# print(predicted_class_indices)
# print(labels)
    predictions = [labels[k] for k in pred1[0]]
    print(predictions)

['Parasitized']
['Parasitized']
['Parasitized']
['Uninfected']
['Uninfected']
['Uninfected']
['Uninfected']
['Uninfected']
['Parasitized']
['Parasitized']


In [15]:
def or_pic(pic):
    img = image.load_img(pic,target_size=(150,150))
    return np.expand_dims(image.img_to_array(img),axis=0)
pic = or_pic(input('文件地址'))
print(labels[model.predict_classes(pic)[0][0]])

文件地址/home/zer0/桌面/cell_images/picture/Parasitized/C33P1thinF_IMG_20150619_120838a_cell_222.png
Parasitized
